<a href="https://colab.research.google.com/github/vimuth97/FYP-Brain-Tumor-Classification/blob/main/Basic%20pathology%20model/rejoin_patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import h5py

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def getAllImageNames(root_dir):
    files = os.listdir(root_dir)
    fileNames = []
    for file in files:
        file = file.split(".")
        fileNames.append(file[0])
    return fileNames

def getAllPathsForImage(name,patches_dir):
    files = os.listdir(patches_dir)
    patchSet = []
    for file in files:
        fileName = file.split(".")
        fileName = fileName[0].split("-")
        if fileName[0] == name:
            patchSet.append(file)
    return patchSet

#getAllImageNames('gdrive/My Drive/Data_Directory')
getAllPathsForImage('CPM19_CBICA_AAB_1','gdrive/My Drive/RESULTS_DIRECTORY/patches')

In [ ]:
files = os.listdir('gdrive/My Drive/RESULTS_DIRECTORY/patches') 
allCoordinates = np.empty((0,2), int)
for f in files:
    file_path = os.path.join('gdrive/My Drive/RESULTS_DIRECTORY/patches', f)   
    fileName = f.split(".")
    fileName = fileName[0].split("-")
    coordinates = [int(x) for x in fileName[1].split("_")]
    print(coordinates)
    with h5py.File(file_path,'r') as hdf5_file:
        dset = hdf5_file['coords']
        for name, value in dset.attrs.items():
            print(name, value)